<a href="https://colab.research.google.com/github/hailusong/colab-god-idclass/blob/master/god_idclass_colabtrain_dlib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training: Traing with Dlib to Detect Key Points of ID

**FIRST OF ALL: CHOOSE RUNTIME ENVIRONMENT TYPE TO BE GPU**<br>
Environment variables setup.<br>
**Tensorflow runtime version list** can be found at [here](https://cloud.google.com/ml-engine/docs/tensorflow/runtime-version-list)

In [0]:
DEFAULT_HOME='/content'
TF_RT_VERSION='1.13'
PYTHON_VERSION='3.5'

YOUR_GCS_BUCKET='id-norm'
YOUR_PROJECT='orbital-purpose-130316'

Dlib and dependents

In [0]:
import cv2
import dlib

## Session and Environment Verification (Destination - Local)

Establish security session with Google Cloud

In [0]:
from google.colab import auth
auth.authenticate_user()


################# RE-RUN ABOVE CELLS IF NEED TO RESTART RUNTIME #################

Verify Versions: TF, Python, IPython and prompt_toolkit (these two need to have compatible version), and protoc

In [0]:
import tensorflow as tf
print(tf.__version__)
assert(tf.__version__.startswith(TF_RT_VERSION + '.')), f'tf.__version__ {tf.__version__} not matching with specified TF runtime version env variable {TF_RT_VERSION}'

1.13.1


In [0]:
!python -V
!ipython --version
!pip show prompt_toolkit
!protoc --version

Python 3.6.7
5.5.0
Name: prompt-toolkit
Version: 1.0.15
Summary: Library for building powerful interactive command lines in Python
Home-page: https://github.com/jonathanslenders/python-prompt-toolkit
Author: Jonathan Slenders
Author-email: UNKNOWN
License: UNKNOWN
Location: /usr/local/lib/python3.6/dist-packages
Requires: wcwidth, six
Required-by: jupyter-console, ipython
libprotoc 3.0.0


## Git Sync for any Change in colab-god-idclass 

In [0]:
![ -e {DEFAULT_HOME}/colab-god-idclass ] && git -C {DEFAULT_HOME}/colab-god-idclass pull
![ ! -e {DEFAULT_HOME}/colab-god-idclass ] && git clone --depth=1 https://github.com/hailusong/colab-god-idclass.git {DEFAULT_HOME}/colab-god-idclass

Cloning into '/content/colab-god-idclass'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 26 (delta 8), reused 8 (delta 0), pack-reused 0
Unpacking objects: 100% (26/26), done.


Push the latest pipeline config file to the GCS

In [0]:
!ls -al {DEFAULT_HOME}/colab-god-idclass/configs/{PIPELINE_CONFIG_NAME}.config
!sed 's/..YOUR_GCS_BUCKET./{YOUR_GCS_BUCKET}/g' < {DEFAULT_HOME}/colab-god-idclass/configs/{PIPELINE_CONFIG_NAME}.config > {DEFAULT_HOME}/colab-god-idclass/configs/{PIPELINE_CONFIG_NAME}_processed.config
!gsutil cp {DEFAULT_HOME}/colab-god-idclass/configs/{PIPELINE_CONFIG_NAME}_processed.config \
           gs://{YOUR_GCS_BUCKET}/data_{MODEL_NAME}

-rw-r--r-- 1 root root 5339 Apr  9 16:29 /content/colab-god-idclass/configs/pipeline_ssd_mobilenet_v2.config
Copying file:///content/colab-god-idclass/configs/pipeline_ssd_mobilenet_v2_processed.config [Content-Type=application/octet-stream]...
/ [1 files][  5.2 KiB/  5.2 KiB]                                                
Operation completed over 1 objects/5.2 KiB.                                      


### Checking Your Google Cloud Storage Bucket

In [0]:
!gsutil ls gs://{YOUR_GCS_BUCKET}/data_{MODEL_NAME}
!gsutil ls gs://{YOUR_GCS_BUCKET}/generated

gs://id-norm/data_ssd_mobilenet_v2/label_map.pbtxt
gs://id-norm/data_ssd_mobilenet_v2/model.ckpt.data-00000-of-00001
gs://id-norm/data_ssd_mobilenet_v2/model.ckpt.index
gs://id-norm/data_ssd_mobilenet_v2/model.ckpt.meta
gs://id-norm/data_ssd_mobilenet_v2/pipeline_ssd_mobilenet_v2_processed.config
gs://id-norm/data_ssd_mobilenet_v2/test.record
gs://id-norm/data_ssd_mobilenet_v2/train.record
gs://id-norm/generated/bbox-train-non-id1.csv
gs://id-norm/generated/bbox-train-non-id2.csv
gs://id-norm/generated/bbox-train-non-id3.csv
gs://id-norm/generated/bbox-train-on-dl.csv
gs://id-norm/generated/bbox-train-on-hc.csv
gs://id-norm/generated/bbox-valid-non-id1.csv
gs://id-norm/generated/bbox-valid-non-id2.csv
gs://id-norm/generated/bbox-valid-non-id3.csv
gs://id-norm/generated/bbox-valid-on-dl.csv
gs://id-norm/generated/bbox-valid-on-hc.csv
gs://id-norm/generated/pnts-train-non-id1.csv
gs://id-norm/generated/pnts-train-non-id2.csv
gs://id-norm/generated/pnts-train-non-id3.csv
gs://id-norm/gene

## Setup Training Data and Validation Data

In [0]:
import os
os.chdir(f'{DEFAULT_HOME}/colab-god-idclass')
!pwd

In [0]:
%run dlib\generate_dlibxml.py generate ..\generated\bbox-train-* ..\generated\dlib_train.xml

In [0]:
%run dlib\generate_dlibxml.py generate ..\generated\bbox-valid-* ..\generated\dlib_test.xml

## Start the Training and Evaluation Jobs

### Option 2: Start the Training Job on CoLab

In [0]:
def train_model(name, xml):
  '''
  requires: the model name, and the path to the xml annotations.
  It trains and saves a new model according to the specified
  training options and given annotations

  example @ https://github.com/Luca96/dlib-minified-models/tree/master/face_landmarks:
    options = dlib.shape_predictor_training_options()
    options.tree_depth = 4
    options.nu = 0.1
    options.cascade_depth = 15
    options.feature_pool_size = 800  # or even 1000
    options.num_test_splits = 200  # 150-200 is enough
  '''
  # get the training options
  options = dlib.shape_predictor_training_options()
  options.tree_depth = 4
  options.nu = 0.1
  options.cascade_depth = 15
  options.feature_pool_size = 400
  options.num_test_splits = 50
  options.oversampling_amount = 5
  #
  options.be_verbose = True  # tells what is happening during the training
  options.num_threads = 4    # number of the threads used to train the model

  # finally, train the model
  dlib.train_shape_predictor(xml, name, options)


def measure_model_error(model, xml_annotations):
    '''requires: the model and xml path.
    It measures the error of the model on the given
    xml file of annotations.'''
    error = dlib.test_shape_predictor(xml_annotations, model)
    print("Error of the model: {} is {}".format(model, error))

Start to train


In [0]:
# train a new model with a subset of the ibug annotations
id_keypoints_train_xml = '..\generated\dlib_train.xml'
id_keypoints_test_xml = '..\generated\dlib_test.xml'
id_keypoints_dat = "..\generated\id_keypoints.dat"

# finally train the eye model
train_model(id_keypoints_dat, id_keypoints_train_xml)

# ..and measure the model error on the testing annotations
measure_model_error(id_keypoints_dat, id_keypoints_test_xml)